In [2]:
import pandas as pd
import numpy as np
import requests
import json


In [3]:
LOCAL_DATA_PATH = './context_data/'
DATA_FILE = 'competition_data_final_pqt'
TARGET_FILE = 'public_train.pqt'

### Признаки на основе популяции по гео

In [4]:
data = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}')
data = data.select(['user_id', 'region_name', 'city_name']).to_pandas()

In [7]:
data.head()

,user_id,region_name,city_name
0,45098,Краснодарский край,Краснодар
1,45098,Краснодарский край,Краснодар
2,45098,Краснодарский край,Краснодар
3,45098,Краснодарский край,Краснодар
4,45098,Краснодарский край,Краснодар


In [8]:
# Скачивание статы по городам

geo_stat_url = "https://raw.githubusercontent.com/hflabs/city/master/city.csv"
geo_stat = pd.read_csv(geo_stat_url)

In [9]:
geo_stat.head()

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,fias_level,capital_marker,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year
0,"Респ Адыгея, г Адыгейск",385200.0,Россия,Южный,Респ,Адыгея,NaN,NaN,г,Адыгейск,...,4,0,79403000000,79703000001,107,UTC+3,44.878414,39.190289,12689,1969
1,г Майкоп,385000.0,Россия,Южный,Респ,Адыгея,NaN,NaN,г,Майкоп,...,4,2,79401000000,79701000001,105,UTC+3,44.609827,40.100661,144055,1857
2,г Горно-Алтайск,649000.0,Россия,Сибирский,Респ,Алтай,NaN,NaN,г,Горно-Алтайск,...,4,2,84401000000,84701000001,400,UTC+7,51.958103,85.960324,62861,1830
3,"Алтайский край, г Алейск",658125.0,Россия,Сибирский,край,Алтайский,NaN,NaN,г,Алейск,...,4,0,1403000000,1703000001,2201,UTC+7,52.492251,82.779361,28528,1913
4,г Барнаул,656000.0,Россия,Сибирский,край,Алтайский,NaN,NaN,г,Барнаул,...,4,2,1401000000,1701000001,2200,UTC+7,53.347997,83.779806,635585,1730


In [10]:
# Выделяем уникальные названия регионов в датафрейме конкурса и в файле со статой
# Так как названия регионов отличаются между файлами, то переименовываю их на единый манер
# в файле со статой

geo_stat_regions = geo_stat.region.unique()
df_regions = data.region_name.unique()

In [11]:
# Вот тут само перименовывание

for geo in geo_stat_regions:
    true_name = list(filter(lambda x: geo in x, df_regions))
    if len(true_name) > 0:
        geo_stat.loc[geo_stat.region==geo, 'region'] = true_name[0]

In [12]:
geo_stat = geo_stat[['region', 'federal_district', 'city', 'population', 'capital_marker']]
geo_stat.rename(columns={'city': 'city_name', 'region': 'region_name'}, inplace=True)
geo_stat = geo_stat.drop_duplicates(subset=['city_name'], keep='first')

In [13]:
# Добавление статы по городам к основному df

data = data.merge(geo_stat[['federal_district', 'city_name', 'population', 'capital_marker']],
                  how='left', on='city_name')

In [14]:
# Добавление статы по регионам к основному df

geo_stat_gr = geo_stat.groupby('region_name', as_index=False)[['population']].sum()
geo_stat_gr.columns = ['region_name', 'region_population']
data = data.merge(geo_stat_gr, how='left', on='region_name')

In [15]:
data.head()

,user_id,region_name,city_name,federal_district,population,capital_marker,region_population
0,45098,Краснодарский край,Краснодар,Южный,744933.0,2.0,2617631.0
1,45098,Краснодарский край,Краснодар,Южный,744933.0,2.0,2617631.0
2,45098,Краснодарский край,Краснодар,Южный,744933.0,2.0,2617631.0
3,45098,Краснодарский край,Краснодар,Южный,744933.0,2.0,2617631.0
4,45098,Краснодарский край,Краснодар,Южный,744933.0,2.0,2617631.0


In [16]:
gc.collect()

24

In [17]:
# Подсчет агрегатов по числовым метрикам

data = pa.Table.from_pandas(data)

df_users_features = data.select(['user_id', 'population', 'capital_marker', 'region_population']).\
                    group_by(['user_id']).aggregate([('population', "mean"),
                                                     ('capital_marker', "mean"),
                                                     ('region_population', "mean")]).to_pandas()

In [18]:
# Подсчет агрегатов по имени федерального округа

fo = data.select(['user_id', 'federal_district']).\
                    group_by(['user_id', 'federal_district']).aggregate([('federal_district', "count")]).to_pandas()
fo['federal_district_count_max'] = fo.groupby(['user_id'], as_index=False)["federal_district_count"].transform('max')
fo = fo.loc[fo.federal_district_count==fo.federal_district_count_max]

df_users_features = df_users_features.merge(fo[['user_id', 'federal_district']], how='left')

In [19]:
# Сохраняем фрейм с фичами по популяционной стате по пользователям

df_users_features.to_csv('users_population_features.csv', index=False)
df_users_features.head()

,population_mean,capital_marker_mean,region_population_mean,user_id,federal_district
0,742648.178710,1.997419,2.611344e+06,45098,Южный
1,NaN,NaN,NaN,117132,None
2,431491.000000,2.000000,7.181590e+05,79395,Северо-Западный
3,241788.000000,0.000000,2.617631e+06,91294,Южный
4,586844.134185,2.000000,8.298867e+05,161323,Уральский


In [20]:
gc.collect()

0

### Признаки выходных дней

In [4]:
data = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}')
data = data.select(['user_id', 'request_cnt', 'date']).to_pandas()

In [5]:
data['date'] = pd.to_datetime(data['date'])

In [6]:
# Не проверял этот календарь, но надеюсь, в нем все корректно)
# Тут даты во фрейме бинарно маркируются: выходной, не выходной

url21 = 'https://raw.githubusercontent.com/d10xa/holidays-calendar/master/json/consultant' + '2021' + '.json'
r21 = requests.get(url21)
cal21 = json.loads(r21.text)

url22 = 'https://raw.githubusercontent.com/d10xa/holidays-calendar/master/json/consultant' + '2022' + '.json'
r22 = requests.get(url22)
cal22 = json.loads(r22.text)

def is_holiday(date):
    if date.year == 2021:
        return cal21['holidays'].count(str(date)[0:-9])
    else:
        return cal22['holidays'].count(str(date)[0:-9])

data['date'] = data['date'].apply(lambda date: is_holiday(date))
data.head()

,user_id,request_cnt,date
0,45098,1,0
1,45098,1,1
2,45098,1,1
3,45098,1,0
4,45098,1,0


In [7]:
data['date'].value_counts()

0    236903363
1     85996072
Name: date, dtype: int64

In [8]:
# Подсчет долей выходных долей по пользовтаелям

data['date'] = data['date'].astype('category')

df = data.groupby(['user_id','date'],group_keys=False)['request_cnt'].sum().reset_index()
df['request_cnt_sum'] = pd.Series(np.repeat(df.groupby('user_id')['request_cnt'].sum().reset_index(drop=True).values,2,axis=0))
df['holyday_fraction'] = df['request_cnt']/df['request_cnt_sum']
df['holyday_fraction'] = df['holyday_fraction'].astype('float32')

df = df.loc[df['date']==1].drop(['request_cnt_sum','request_cnt','date'],axis=1).reset_index(drop=True)

In [9]:
# Сохранение

df.to_csv('users_holyday_fraction.csv', index=False)
df.head()

,user_id,holyday_fraction
0,0,0.243523
1,1,0.200573
2,2,0.299270
3,3,0.181818
4,4,0.326898


In [10]:
gc.collect()

0

### Тут считаются фичи доли мужских сайтов

In [ ]:
data = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}')
data = data.select(['user_id', 'request_cnt', 'url_host']).to_pandas()

In [ ]:
# Из датафрейма удалются сайты, у которых было менее 50 уникальных пользователей

sites = (data.groupby('url_host', as_index=False)
           .agg(request_cnt=('request_cnt', 'sum'), unique_user_id=('user_id', 'nunique'))
           .sort_values(by='request_cnt', ascending=False))

drop_sites_50 = list(sites.loc[sites.unique_user_id<=50].url_host.unique())

data = data.loc[~data['url_host'].isin(drop_sites_50)]

In [6]:
# Подгружаются таргеты по полу
# Для каждого сайта считается число запросов, число запросов мужчин
# и доля от этих двух агрегатов

#target = pd.read_feather('/kaggle/input/mts-ml-cookies/target_train.feather', columns=['user_id','is_male'])
target = pq.read_table(f'{LOCAL_DATA_PATH}/{TARGET_FILE}').to_pandas()[['user_id', 'is_male']]

target['user_id'] = target['user_id'].astype('int32')
target['is_male'] = target['is_male'].astype('object')

data = data.merge(target[['is_male','user_id']], on = 'user_id', how = 'inner')
data = data.loc[~(data['is_male'].isna()) & (data['is_male'] != 'NA')]
data['is_male'] = data['is_male'].astype('int8')

data['male_request_cnt'] = data['is_male'] * data['request_cnt']
df = data.groupby('url_host', as_index=False).agg({'request_cnt':'sum','male_request_cnt':'sum'})
df['male_fraction'] = df['male_request_cnt'] / df['request_cnt']

In [7]:
df.to_csv('sites_male_fraction.csv', index=False)
df.head()

,url_host,request_cnt,male_request_cnt,male_fraction
0,-1,27886,24157,0.866277
1,0-hi--tech-mail-ru-0.cdn.ampproject.org,4296,2801,0.652002
2,003ms.ru,438,181,0.413242
3,010203.org,69,36,0.521739
4,0370.ru,124,31,0.250000


In [5]:
df = pd.read_csv('./context_data/custom_data/sites_male_fraction.csv')

In [6]:
# Заново загружается основной фрейм, так как выше он редактировался

data = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}')
#data = data.select(['user_id', 'request_cnt', 'url_host']).to_pandas()

In [7]:
# К сгруппированному основному фрейму мержится датафрейм с male_fraction из ячеек выше
# Далее группировка по пользователям

#data = pa.Table.from_pandas(data)
data = data.group_by(['user_id', 'url_host']).aggregate([('request_cnt', "sum")]).to_pandas()

In [8]:

data = data.merge(df[['url_host', 'male_fraction']], how='left')

In [9]:

data['male_fraction_cum'] = data['male_fraction'] * data['request_cnt_sum']

users_male_fraction = data.groupby(['user_id'], as_index=False)[['male_fraction', 'male_fraction_cum']].mean()

In [10]:
users_male_fraction.to_csv('users_male_fraction.csv', index=False)
users_male_fraction.head()

,user_id,male_fraction,male_fraction_cum
0,0,0.474666,1.857592
1,1,0.511873,4.114327
2,2,0.529900,4.238684
3,3,0.512286,3.386407
4,4,0.482122,3.703606


In [11]:
gc.collect()

37

### Тут считаются агрегаты по возрасту

In [4]:
# Загрузка основного фрейма, удаление сайтов с менее 50 уников
# Примердживание таргета по возрасту


data = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}')\
    .select(['user_id', 'request_cnt', 'url_host']).to_pandas()

sites = (data.groupby('url_host', as_index=False)
           .agg(request_cnt=('request_cnt', 'sum'), unique_user_id=('user_id', 'nunique'))
           .sort_values(by='request_cnt', ascending=False))

drop_sites_50 = list(sites.loc[sites.unique_user_id<=50].url_host.unique())

data = data.loc[~data['url_host'].isin(drop_sites_50)]

target = pq.read_table(f'{LOCAL_DATA_PATH}/{TARGET_FILE}').to_pandas()[['user_id', 'age']]
target['user_id'] = target['user_id'].astype('int32')

# добавляем target, удаляем nan
data = data.merge(target[['age','user_id']], on = 'user_id', how = 'inner')
data = data.loc[~(data['age'].isna()) & (data['age'] > 18) & (data['age'] != 'NA')]
data['age'] = data['age'].astype('int16')

In [5]:
# Группировка по url и подсчет агрегатов по возрасту

def q10(x): return x.quantile(0.1)
def q25(x): return x.quantile(0.25)
def q75(x): return x.quantile(0.75)
def q90(x): return x.quantile(0.9)

data = data[['user_id', 'url_host', 'age']].drop_duplicates(subset=['user_id', 'url_host'], keep='first')
df = data.groupby('url_host', as_index=False).agg(median_age=('age', 'median'),
                                                  q10_age=('age', q10),
                                                  q25_age=('age', q25),
                                                  q75_age=('age', q75),
                                                  q90_age=('age', q90),
                                                  avg_age=('age', 'mean'))

In [6]:
data = data[['user_id', 'url_host', 'age']].drop_duplicates(subset=['user_id', 'url_host'], keep='first')
df_new_feat = data.groupby('url_host', as_index=False).agg(
    min_age = ('age', 'min'),
    max_age = ('age', 'max'),
    q5_age = ('age', lambda x: x.quantile(.05)),
    q15_age = ('age', lambda x: x.quantile(.15)),
    q20_age = ('age', lambda x: x.quantile(.20)),
    q30_age = ('age', lambda x: x.quantile(.30)),
    q35_age = ('age', lambda x: x.quantile(.35)),
    q40_age = ('age', lambda x: x.quantile(.40)),
    q45_age = ('age', lambda x: x.quantile(.45)),
    q55_age = ('age', lambda x: x.quantile(.55)),
    q60_age = ('age', lambda x: x.quantile(.60)),
    q65_age = ('age', lambda x: x.quantile(.65)),
    q70_age = ('age', lambda x: x.quantile(.70)),
    q80_age = ('age', lambda x: x.quantile(.80)),
    q85_age = ('age', lambda x: x.quantile(.85)),
    q95_age = ('age', lambda x: x.quantile(.95)),
    std_age = ('age', 'std'),
    mode_age = ('age', pd.Series.mode)
)

In [ ]:
df.to_csv('sites_age_distribution.csv', index=False)
df.head()

In [8]:
df_new_feat.to_csv('sites_age_distribution_2.csv', index=False)


In [13]:
# Снова подгрузка основного фрейма и джоин фрейма выше с агрегатами возрастов
# Группировка по пользователям


data = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}')\
    .select(['user_id', 'request_cnt', 'url_host'])

data = data.group_by(['user_id', 'url_host']).aggregate([('request_cnt', "sum")]).to_pandas()
data = data.merge(df_new_feat,
                  how='left')

users_age_distribution = data.groupby(['user_id'], as_index=False).mean()

In [16]:
users_age_distribution = users_age_distribution.drop('request_cnt_sum', axis=1)

In [17]:
users_age_distribution.to_csv('users_age_distribution_2.csv', index=False)
users_age_distribution.head()

,user_id,min_age,max_age,q5_age,q15_age,q20_age,q30_age,q35_age,q40_age,q45_age,q55_age,q60_age,q65_age,q70_age,q80_age,q85_age,q95_age,std_age
0,0,19.000000,84.576923,22.750000,25.975962,27.634615,30.601923,31.923077,33.230769,34.601923,37.557692,39.019231,40.711538,42.457692,47.696154,50.605769,59.095192,11.207079
1,1,19.203008,84.210526,23.917293,27.700752,29.410526,32.434586,33.847368,35.309774,36.717669,39.662030,41.267669,43.130827,45.008271,49.912782,52.614662,60.446241,11.318622
2,2,19.019608,88.254902,22.607843,25.713725,27.337255,29.980392,31.345098,32.694118,33.905882,36.662745,38.129412,39.803922,41.588235,46.352941,49.215686,58.043137,10.942105
3,3,19.142857,88.619048,22.738095,25.911905,27.509524,30.261905,31.619048,32.980952,34.188095,36.952381,38.361905,40.047619,41.738095,46.476190,49.421429,58.426190,10.955447
4,4,19.018692,85.336449,22.645794,25.728037,27.308411,29.935514,31.192056,32.504673,33.699533,36.414953,37.805607,39.361215,41.093458,45.626168,48.516355,57.512150,10.718785


In [8]:
# Снова подгрузка основного фрейма и джоин фрейма выше с агрегатами возрастов
# Группировка по пользователям


data = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}')\
    .select(['user_id', 'request_cnt', 'url_host'])

data = data.group_by(['user_id', 'url_host']).aggregate([('request_cnt', "sum")]).to_pandas()
data = data.merge(df[['url_host', 'median_age', 'q10_age', 'q25_age', 'q75_age', 'q90_age', 'avg_age']],
                  how='left')

users_age_distribution = data.groupby(['user_id'], as_index=False)[['median_age', 'q10_age', 'q25_age',
                                                                    'q75_age', 'q90_age', 'avg_age']].mean()

In [9]:
users_age_distribution.to_csv('users_age_distribution.csv', index=False)
users_age_distribution.head()

,user_id,median_age,q10_age,q25_age,q75_age,q90_age,avg_age
0,0,36.028846,24.301923,29.076923,44.730769,54.180769,37.801677
1,1,38.161654,25.871429,30.962406,47.184211,55.955639,39.708257
2,2,35.215686,24.058824,28.647059,43.705882,53.035294,37.098618
3,3,35.404762,24.176190,28.892857,43.857143,53.280952,37.303979
4,4,34.948598,24.196262,28.565421,43.116822,52.268224,36.800544


In [10]:
gc.collect()

1080